<a href="https://colab.research.google.com/github/JLMR-Code-Creator/NELDA/blob/main/DeepGA_NELDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio torchtext
!pip install pytorch-forecasting
!pip install mlxtend
!pip install torchmetrics
!pip install medmnist

In [2]:
import medmnist
import torch.utils.data as data
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.mixture import GaussianMixture
from medmnist import INFO, Evaluator
import torch.utils.data as data
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.mixture import GaussianMixture

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
 %cd /content/drive/MyDrive/NELDA

/content/drive/MyDrive/NELDA


# DeepGA is an algorithm for evolve a convolutional neural network.


In [5]:
from DeepGA.Operators import *
from DeepGA.EncodingClass import Encoding
from DeepGA.Decoding import *
from DeepGA.DataReader import *
from DeepGA.DistributedTraining import *
from DeepGA.DeepGA import *

# Data reader

In [6]:
def loading_data():
# preprocessing
    print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

    #flag = "pathmnist"
    info_pathmnist = INFO['pathmnist']
    DataClass_pathmnist = getattr(medmnist, info_pathmnist['python_class'])
    data_transform = transforms.Compose([
       transforms.ToTensor(),
    ])
    train_data = DataClass_pathmnist(split='train', transform=data_transform, download=True)
    val_data = DataClass_pathmnist(split='val', transform=data_transform, download=True)
    test_data = DataClass_pathmnist(split='test', transform=data_transform, download=True)

    BATCH_SIZE = 128
    train_dl = data.DataLoader(train_data, batch_size= BATCH_SIZE, shuffle = True)
    val_dl = data.DataLoader(val_data, batch_size= BATCH_SIZE, shuffle = False)
    test_dl = data.DataLoader(test_data, batch_size= BATCH_SIZE, shuffle = False)

    data_flag = 'pathmnist'
    info = INFO[data_flag]
    task = info['task']
    n_channels = info['n_channels']
    n_classes = len(info['label'])

    return train_dl, val_dl, test_dl, n_channels, n_classes, 28

### Call DeepGA for evolve CNN´s

In [7]:
'''Defining DeepGA hyperparameters'''

#Convolutional layers
FSIZES = [3, 5, 7, 9] # Odd Sizes Are Preferred
NFILTERS = [8, 16, 32, 64, 128, 256]

#Pooling layers
PSIZES = [2,3] #[2,3,4,5]
PTYPE = ['max', 'avg']

#Fully connected layers
NEURONS = [16, 32, 64, 128, 256] # for big layers

#Defining learning rate
lr = 1e-4

#Maximun and minimum numbers of layers to initialize networks
min_conv = 2 # 30
max_conv = 30 # 60
min_full = 1
max_full = 3 # 10
max_params = 5e6
train_epochs = 5 # Epochs to train the best individual found by the GA

'''Genetic Algorithm Parameters'''
cr = 0.7   # Crossover rate
mr = 0.5   # Mutation rate
N = 20      # Population size 20 Se mantuvo en 20
T = 30      # Number of generations
t_size = 5 # tournament size 5
w = 0.1    # penalization weight   0.3
#chck_dir = '/content/drive/MyDrive/checkpoint/'  # Root folder for dumping/loading pickle
chck_dir = 'point/'  # Root folder for dumping/loading pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dl, val_dl, test_dl, n_channels, n_classes, out_size = loading_data() #Loading data

loss_func = nn.CrossEntropyLoss()
execution_ID = 707 #Execution ID for checkpoint, change it for a new execution
results, pop, bestind  = deepGA(execution_ID, True, train_epochs = train_epochs, train_dl=train_dl, val_dl=val_dl,  lr=lr,
                       min_conv=min_conv, max_conv=max_conv, min_full=min_full, max_full=max_full, max_params=max_params,
                       cr=cr, mr=mr, N=N, T=T, t_size=t_size, w=w, device=device, chck_dir=chck_dir,
                       n_channels =  n_channels , n_classes=n_classes, out_size = out_size, loss_func=loss_func)

# training to more epochs

finalEpochs = 100
CNNModel = final_evaluation(execution_ID, bestind, train_dl, val_dl, lr, max_params, w, device, finalEpochs, loss_func, chck_dir, n_channels =  n_channels , n_classes=n_classes, out_size = out_size)

MedMNIST v3.0.2 @ https://github.com/MedMNIST/MedMNIST/


100%|██████████| 206M/206M [00:12<00:00, 15.9MB/s]


Re-Initialize population
The maximum number of generations has been reached. Please run a new execution.
--------------------------------------------
    Accuracy   Fitness  No. Params   MeanFit   MeanAcc    MeanPar
0   0.917033  0.914662      533393  0.850344  0.865204  1416999.4
1   0.917033  0.914662      533393  0.888973  0.895127   832081.8
2   0.912435  0.918521      133529  0.904107  0.905838   557350.2
3   0.919932  0.919114      441217  0.911085  0.911650   470007.0
4   0.919932  0.919114      441217  0.913154  0.913500   449759.0
5   0.919932  0.919114      441217  0.913784  0.913929   437624.2
6   0.919932  0.919114      441217  0.916064  0.915109   376713.4
7   0.919932  0.919114      441217  0.916607  0.916054   392084.2
8   0.917033  0.922630      135001  0.917481  0.916553   370829.4
9   0.924930  0.923736      435041  0.918805  0.917248   335918.6
10  0.924930  0.923736      435041  0.919436  0.917368   309762.2
11  0.924930  0.923736      435041  0.919741  0.917288   2

### Building NELDA models

In [13]:
def extract_features(feature_extractor, loader):
    features = []
    labels = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    feature_extractor = feature_extractor.to(device)
    with torch.no_grad():
        for images, targets in loader:
            images = images.to(device)
            outputs = feature_extractor(images).squeeze()  # Remove singleton dimensions
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

def predictions(feature_extractor_CNN_Model, LDA_model, data):
    features, labels_test = extract_features(feature_extractor_CNN_Model, data) # Getting features of test images
    predicted_labels = LDA_model.predict(features)
    probabilities = LDA_model.predict_proba(features)
    reduced_features = LDA_model.transform(features) # LDA reduction of test features, are projected onto a lower-dimensional space
    return probabilities, predicted_labels, reduced_features, labels_test

def evaluate(feature_extractor_CNN_Model, LDA_model, data_dl):
    probabilities, predicted_labels, reduced_features, targets = predictions(feature_extractor_CNN_Model, LDA_model, data_dl)
    idx_incorrectly_classified = []
    idx_correctly_classified = []
    # Identify incorrectly classified samples based on a probability threshold
    threshold = 0.9817
    for i in range(len(probabilities)):
        prob = probabilities[i]
        if np.max(prob) < threshold:
           idx_incorrectly_classified.append(i)
        else:
            idx_correctly_classified.append(i)
    #get labels of correctly classified samples based on the threshold
    labels_correctly_classified = [predicted_labels[i] for i in idx_correctly_classified]
    # get the amount of labels groups in correctly classified samples
    unique_labels, counts = np.unique(labels_correctly_classified, return_counts=True)
    print("Label counts in correctly classified samples:", str(len(unique_labels)))
    n_labels_found = len(unique_labels)
    if(n_labels_found < 6):
        missing_classes = 6 - n_labels_found # Assuming we want a total of 6 classes
        data_incorrectly_classified = [reduced_features[i] for i in idx_incorrectly_classified]
        gmm = GaussianMixture(n_components=missing_classes, init_params='kmeans', covariance_type='full')#####<-------------
        gmm.fit(data_incorrectly_classified)
        gmm_labels = gmm.predict(data_incorrectly_classified)
        newLabels = gmm_labels + 100 # Offset to avoid label conflicts
        # Append new classes to the original labels
        labels_test_extended = np.copy(predicted_labels)
        for i, idx in enumerate(idx_incorrectly_classified):
            labels_test_extended[idx] = newLabels[i]
        print("New classes assigned to incorrectly classified samples:", np.unique(newLabels))
    else:
        labels_test_extended = predicted_labels
        print("No new classes assigned, all classes are present.")
    return labels_test_extended, targets

In [9]:
CNNModel.eval()
DeepGA_CNN_Model = copy.deepcopy(CNNModel)
DeepGA_CNN_Model.eval()  # Set the model to evaluation mode
# Remove the final classification layer to get features
DeepGA_CNN_Model.classifier = nn.Sequential(*list(DeepGA_CNN_Model.classifier.children())[:-1])
DeepGA_CNN_Model.classifier = nn.Sequential(*list(DeepGA_CNN_Model.classifier.children())[:-2])
feature_extractor_CNN_Model = DeepGA_CNN_Model
feature_extractor_CNN_Model.eval()

features_train, labels_train = extract_features(feature_extractor_CNN_Model, train_dl)
#LDA Classifier
LDA_model = LinearDiscriminantAnalysis(solver='eigen', n_components=2)
LDA_model.fit(features_train, labels_train)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearDiscriminantAnalysis(n_components=2, solver='eigen')

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


labels_train_extended, targets_train = evaluate(feature_extractor_CNN_Model, LDA_model, train_dl)
accuracy = accuracy_score(targets_train, labels_train_extended)
print(f'Accuracy of the LDA model on the train set: {accuracy * 100:.2f}%')

labels_val_extended, targets = evaluate(feature_extractor_CNN_Model, LDA_model, val_dl)
accuracy = accuracy_score(targets, labels_val_extended)
print(f'Accuracy of the LDA model on the val set: {accuracy * 100:.2f}%')

labels_test_extended, targets = evaluate(feature_extractor_CNN_Model, LDA_model, test_dl)
accuracy = accuracy_score(targets, labels_test_extended)
print(f'Accuracy of the LDA model on the test set: {accuracy * 100:.2f}%')

Label counts in correctly classified samples: 9
No new classes assigned, all classes are present.
Accuracy of the LDA model on the train set: 99.86%
Label counts in correctly classified samples: 9
No new classes assigned, all classes are present.
Accuracy of the LDA model on the val set: 95.52%
Label counts in correctly classified samples: 9
No new classes assigned, all classes are present.
Accuracy of the LDA model on the test set: 86.31%


In [15]:
from pathlib import Path
import copy
import zipfile
import joblib

# Prepare output dir for DeepGA andONNX models
out_dir = Path('models/ensemble')
out_dir.mkdir(parents=True, exist_ok=True)

export_model = copy.deepcopy(feature_extractor_CNN_Model)

#dummy_shape = infer_dummy_shape(export_model)
wrapped = export_model

wrapped.eval()
wrapped.to('cpu')
#dummy_input = torch.randn(*dummy_shape)

cnn_joblib_path = out_dir / 'DeepGA.joblib'
joblib.dump(wrapped, cnn_joblib_path)
print('Saved DeepGA Model to', wrapped)


if 'LDA_model' not in globals():
    raise RuntimeError('LDA_model not found')


lda_model_path = out_dir / 'lda.joblib'

if 'LDA_model' in globals():
    joblib.dump(LDA_model, lda_model_path)
    print('Saved LDA model to', lda_model_path)
else:
    print('No LDA model found; skipping')

gmm_path = out_dir / 'gmm.joblib'
if 'gmm' in globals():
    joblib.dump(gmm, gmm_path)
    print('Saved GMM to', gmm_path)
else:
    print('No GMM found; skipping')

zip_path = out_dir.parent / 'ensemble.zip'
with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    if cnn_joblib_path.exists():
        zf.write(cnn_joblib_path, arcname='DeepGA.joblib')
    if lda_model_path.exists():
        zf.write(lda_model_path, arcname='lda.jolib')
    if gmm_path.exists():
        zf.write(gmm_path, arcname='gmm.joblib')
print('Created', zip_path)

print('Export and packaging complete')


Saved DeepGA Model to CNN(
  (extraction): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(320, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (